In [1]:
import os
import gc
import optuna
import warnings
import numpy as np
from utils import *
import pandas as pd
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
warnings.filterwarnings('ignore')
%run skills.ipynb
%run languages.ipynb
%run work_experiences.ipynb
%run education.ipynb

class config:
    train_path = '../../../datasets/garanti-bbva-data-camp/train_users.csv'
    test_path = '../../../datasets/garanti-bbva-data-camp/test_users.csv'
    sub_path = '../../../datasets/garanti-bbva-data-camp/submission.csv'
    skills_path = '../../../datasets/garanti-bbva-data-camp/skills.csv'
    languages_path = '../../../datasets/garanti-bbva-data-camp/languages.csv'
    education_path = '../../../datasets/garanti-bbva-data-camp/education.csv'
    exp_path = '../../../datasets/garanti-bbva-data-camp/work_experiences.csv'
    seed = 42
    n_folds = 6

In [2]:
train_df = pd.read_csv(config.train_path)
test_df = pd.read_csv(config.test_path)
sub = pd.read_csv(config.sub_path)

print(f'train_df shape: {train_df.shape}')
print(f'test_df shape: {test_df.shape}')
df = train_df.append(test_df).reset_index(drop = True)
df.head()

train_df shape: (53019, 4)
test_df shape: (13255, 3)


,user_id,industry,location,moved_after_2019
0,1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1.0
1,6950,Internet,"Istanbul, Istanbul, Turkey",0.0
2,4880,Online Media,Turkey,0.0
3,26046,Telecommunications,"Istanbul, Istanbul, Turkey",0.0
4,11005,Banking,"Istanbul, Turkey",0.0


In [3]:
def fix_location(dataframe: pd.DataFrame) -> pd.DataFrame:

    df_ = dataframe.copy()
    df_.loc[df_['location'].astype(str).str.contains('Kahraman Maras'), 'location'] = 'Kahramanmaras, Turkey'
    df_.loc[df_['location'].astype(str).str.contains('Şanliurfa'), 'location'] = 'Sanliurfa, Turkey'
    df_.loc[df_['location'].astype(str).str.contains('İçel'), 'location'] = 'Mersin, Turkey'
    df_.loc[df_['location'].astype(str).str.contains('Afyon'), 'location'] = 'Afyonkarahisar, Turkey'
    df_['location'] = df_['location'].apply(lambda x: str(x).replace('Türkiye', 'Turkey'))
    df_['location'] = df_['location'].apply(lambda x: x.upper().strip())
    df_['location'] = df_['location'].apply(lambda x: translation(str(x)))
    tr_cities = load_tr_cities()
    for city in tr_cities:
        df_['location'] = df_['location'].apply(lambda x: city if city in x else x)
        df_['based_on_tr'] = df_['location'].apply(lambda x: 1 if x in tr_cities or x == 'TURKEY' else 0)

    return df_

df = fix_location(df)
df.head()

,user_id,industry,location,moved_after_2019,based_on_tr
0,1301,Information Technology and Services,ISTANBUL,1.0,1
1,6950,Internet,ISTANBUL,0.0,1
2,4880,Online Media,TURKEY,0.0,1
3,26046,Telecommunications,ISTANBUL,0.0,1
4,11005,Banking,ISTANBUL,0.0,1


In [4]:
def objective(trial):

    df_ = df.copy()
    skill_size = trial.suggest_int('skill_size', 10, 60)
    lang_size = trial.suggest_int('lang_size', 4, 20)
    study_size = trial.suggest_int('study_size', 4, 40)
    degree_size = trial.suggest_int('degree_size', 4, 20)

    skills_df = load_skills(config.skills_path, skill_size)
    lang_df = load_languages(config.languages_path, lang_size)
    edu_df = load_education(config.education_path, study_size, degree_size)
    exp_df = load_work_experiences(config.exp_path)

    df_ = df_.merge(skills_df, on = ['user_id'], how = 'left')
    df_ = df_.merge(lang_df, on = ['user_id'], how = 'left')
    df_ = df_.merge(edu_df, on = ['user_id'], how = 'left')
    df_ = df_.merge(exp_df, on = ['user_id'], how = 'left')
    df_['nunique_company_by_industries'] = df_.groupby(by = 'industry')['company_id'].transform('nunique')

    target = 'moved_after_2019'
    cat_features = ['industry', 'location', 'company_id']
    drop_features = ['user_id']

    for caterogical_col in cat_features:
        df_[caterogical_col] = df_[caterogical_col].astype(str).astype("category")

    train_set = df_.loc[df_[target].notnull()].drop(columns=drop_features, axis = 1)
    test_set = df_.loc[df_[target].isnull()].drop(columns=drop_features, axis = 1)

    X = train_set.drop(columns = [target], axis = 1)
    y = train_set[target]

    params = {
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 100, 220),
        "depth": trial.suggest_int("depth", 5, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    params['eval_metric'] = 'Accuracy'
    params['cat_features'] = cat_features
    params['random_state'] = config.seed
    params['allow_writing_files'] = False
    params["iterations"] = 5000
    kf = KFold(n_splits=config.n_folds)

    scores = list()
    for idx, (train_ind, val_ind) in enumerate(kf.split(X, y)):
        print(f'fold: {idx+1}')
        model = CatBoostClassifier(**params)
        X_train = X.iloc[train_ind]
        y_train = y.iloc[train_ind]
        X_val = X.iloc[val_ind]
        y_val = y.iloc[val_ind]

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=400,
            verbose=False,
        )

        val_pred = model.predict(X_val)
        scores.append(accuracy_score(y_val, val_pred))
        del X_train, y_train, X_val, y_val
        gc.collect()

    return np.mean(scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)

[I 2023-02-08 03:07:32,231] A new study created in memory with name: no-name-ff7ba42e-4ceb-4971-8be3-0af0339461f9


Frequency of top 20 skills before preprocess: 294433
Frequency of top 20 skills after preprocess: 320842


100%|██████████| 13/13 [00:03<00:00,  4.17it/s]


fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-08 04:31:31,956] Trial 0 finished with value: 0.7672720825724194 and parameters: {'skill_size': 52, 'lang_size': 7, 'study_size': 20, 'degree_size': 13, 'one_hot_max_size': 188, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.6217375982247557}. Best is trial 0 with value: 0.7672720825724194.


...

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-08 11:16:27,988] Trial 19 finished with value: 0.7776832866202313 and parameters: {'skill_size': 40, 'lang_size': 13, 'study_size': 23, 'degree_size': 19, 'one_hot_max_size': 176, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 5 with value: 0.7815689060558735.


Frequency of top 20 skills before preprocess: 294433


[W 2023-02-08 11:16:46,433] Trial 20 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/67/wq_xsymd3jvc5w2fx1ld_18h0000gn/T/ipykernel_86990/660928086.py", line 9, in objective
    skills_df = load_skills(config.skills_path, skill_size)
  File "/var/folders/67/wq_xsymd3jvc5w2fx1ld_18h0000gn/T/ipykernel_86990/3064544183.py", line 245, in load_skills
    df_.loc[df_['skill'] == 'Postgresql', 'skill'] = 'PostgreSQL'
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py", line 723, in __setitem__
    iloc._setitem_with_indexer(indexer, value, self.name)
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py", line 1730, in _setitem_with_indexer
    self._setitem_with_indexer_split_path(indexer,

KeyboardInterrupt: 

In [5]:
study.best_params

{'skill_size': 60,
 'lang_size': 14,
 'study_size': 21,
 'degree_size': 15,
 'one_hot_max_size': 156,
 'depth': 12,
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS'}

{'skill_size': 48, 'lang_size': 10, 'study_size': 9, 'degree_size': 11}